In [1]:
# imports
import requests
import json
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [6]:
# connect to foursquare api

def fs_api_request(ll,radius,categories,fields):
    fs_url = 'https://api.foursquare.com/v3/places/search' 
    key = os.environ.get('FOURSQUARE_API_KEY')
    fs_headers = {"authorization": f'{key}',"accept": "application/json"}
    fs_params = {
        "ll": ll,
        "radius": radius,
        "categories": categories,
        "fields": fields,
    }
    fs_response = requests.get(fs_url, headers=fs_headers, params=fs_params)
    fs_data = fs_response.json()

    return fs_data

bike_df = pd.read_csv('bike_stations.csv') # read bike stations csv

for index,row in bike_df.iterrows():
    ll = f"{row['latitude']},{row['longitude']}"
    results = fs_api_request(ll,radius=1000,categories=None,fields=None)
print(results['results'][0]['location']['formatted_address'])

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
for index,row in bike_df.iterrows():
    ll = f"{row['latitude']},{row['longitude']}"
    results = fs_api_request(ll,radius=1000,categories=None,fields=None)

    venue_name = results['results'][0]['categories'][0]['name']
    venue_location = results['results'][0]['location']['formatted_address']

    print(f'name = {venue_name}, location = {venue_location}')

Put your parsed results into a DataFrame

In [ ]:
# create dataframe to store venue data
fs_venue_df = pd.DataFrame()

for index,row in bike_df.iterrows():
    ll = f"{row['latitude']},{row['longitude']}"
    results = fs_api_request(ll,radius=1000,categories=13065,fields=None) # 13065 = restaurant
    norm_results = pd.json_normalize(results)
    norm_results['station_id'] = row['id'] # add station id to dataframe

    fs_venue_df = pd.concat([fs_venue_df,norm_results], ignore_index=True)

fs_venue_df

In [ ]:
fs_venue_df.to_csv('fs_venue_df.csv', index=False) # save dataframe to csv
fs_venue_df

In [ ]:
fs_venue = pd.read_csv('fs_venue_df.csv') # read csv

normalized_results_column = pd.json_normalize(fs_venue['results'])
fs_venue['distance'] = normalized_results_column['distance']

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
# connect to yelp api

def yelp_api_request(latitude,longitude,radius,categories):
    key = os.environ.get('YELP_API_KEY')
    yelp_url = "https://api.yelp.com/v3/businesses/search"
    yelp_headers = {"authorization":f"Bearer {key}","accept": "application/json"}
    yelp_params = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "categories": categories,
        "limit": 10
        }
    yelp_response = requests.get(yelp_url, headers=yelp_headers, params=yelp_params)
    yelp_data = yelp_response.json()

    if 'businesses' in yelp_data:
        return yelp_data['businesses']
    else:
        print(f"Error: {yelp_data}")
        return []

# read bike stations csv
bike_df = pd.read_csv('bike_stations.csv')

for index,row in bike_df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    results = yelp_api_request(latitude,longitude,radius=1000,categories=None)
    print(results)

KeyError: 'YELP_API_KEY'

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
for index,row in bike_df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    results = yelp_api_request(latitude,longitude,radius=1000,categories='restaurants')

    venue_name = results[0]['name']
    venue_location = results[0]['location']

    print(f'name = {venue_name}, location = {venue_location}')

Put your parsed results into a DataFrame

In [ ]:
# create dataframe to store venue data
yelp_venue_df = pd.DataFrame()

for index,row in bike_df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    results = yelp_api_request(latitude,longitude,radius=1000,categories='restaurants')
    norm_results = pd.json_normalize(results)
    norm_results['station_id'] = row['id'] # add station id to dataframe

    yelp_venue_df = pd.concat([yelp_venue_df,norm_results], ignore_index=True)

yelp_venue_df

In [ ]:
# save yepl venue dataframe to csv to avoid maxxing out api calls
yelp_venue_df.to_csv('yelp_venue_df.csv', index=False)

In [215]:
yelp_venues = pd.read_csv('yelp_venue_df.csv')

# Select and rename columns from csv to create new dataframe
new_yelp_df = yelp_venues[['name', 'rating', 'coordinates.longitude', 'coordinates.latitude', 'location.address1', 'station_id']].rename(columns={
    'name': 'Name',
    'rating': 'Rating',
    'coordinates.longitude': 'Longitude',
    'coordinates.latitude': 'Latitude',
    'location.address1': 'Address',
    'station_id': 'station_id'
})

new_yelp_df

# save new yelp dataframe to csv
new_yelp_df.to_csv('new_yelp_df.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

I believe yelp provided me with more complete data, it had a lot more fields available for you to use to decide what information you wanted to include. It was also a cleaner data set to work with. Yelp gave me more options on what information I could extract for the restaurants around the stations. Additionally, Yelp provided me with more POIs (2463) compared to foursquare (247).

Get the top 10 restaurants according to their rating

In [ ]:
new_yelp_df.sort_values(by=['Rating'], ascending=False).head(10)